# Semantic Kernel Tool Use Example - Simulating flight booking

## Import the Needed Packages 

In [1]:
import json
import os

from dotenv import load_dotenv

from IPython.display import display, HTML

from typing import Annotated
from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## Creating the Plugins    
Semantic Kernel uses plugins as tools that can be called by the agent. A plugin can have multiple `kernel_functions` in it as a group. 

In the example below, we create a `FlightSimulation` that has two functions: 
1. Returns a list of available destinations along with their corresponding flight prices `get_flight_prices(origin, destination)`
2. Returns the seat availability for a given route and destination `get_availabilty(origin, destination)`

In [8]:
# Define a sample plugin for flights (same code/signature, richer example data)
class FlightSimulation:
    """Simulates flight search and availability for given destinations."""

    @kernel_function(description="Provides flight prices for a given origin and destination.")
    def get_flight_prices(
        self,
        origin: Annotated[str, "The city or airport of departure."],
        destination: Annotated[str, "The city or airport of arrival."]
    ) -> Annotated[str, "Returns the flight price for the given route."]:
        return """
        New York (JFK) → Tokyo (HND) - $950
        Los Angeles (LAX) → Paris (CDG) - $780
        London (LHR) → Rome (FCO) - $220
        Charlotte (CLT) → Miami (MIA) - $160
        Tokyo (NRT) → Sydney (SYD) - $620
        San Francisco (SFO) → Singapore (SIN) - $880
        Dubai (DXB) → New Delhi (DEL) - $340
        Frankfurt (FRA) → Barcelona (BCN) - $140
        Toronto (YYZ) → New York (LGA) - $120
        Mexico City (MEX) → Madrid (MAD) - $540
        """

    @kernel_function(description="Provides seat availability for a given origin and destination.")
    def get_availability(
        self,
        origin: Annotated[str, "The city or airport of departure."],
        destination: Annotated[str, "The city or airport of arrival."]
    ) -> Annotated[str, "Returns the seat availability for the given route."]:
        return """
        New York (JFK) → Tokyo (HND) - Limited
        Los Angeles (LAX) → Paris (CDG) - Available
        London (LHR) → Rome (FCO) - Unavailable
        Charlotte (CLT) → Miami (MIA) - Available
        Tokyo (NRT) → Sydney (SYD) - Waitlist
        San Francisco (SFO) → Singapore (SIN) - Available
        Dubai (DXB) → New Delhi (DEL) - Limited
        Frankfurt (FRA) → Barcelona (BCN) - Available
        Toronto (YYZ) → New York (LGA) - Available
        Mexico City (MEX) → Madrid (MAD) - Unavailable
        """


## Creating the Client

In this sample, we will use [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for access to the LLM. 

The `ai_model_id` is defined as `gpt-4o-mini`. Try changing the model to another model available on the GitHub Models marketplace to see the different results. 

For us to use the `Azure Inference SDK` that is used for the `base_url` for GitHub Models, we will use the `OpenAIChatCompletion` connector within Semantic Kernel. There are also other [available connectors](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) to use Semantic Kernel for other model providers.

In [9]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.getenv("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Creating the Agent 
Now we will create the Agent by using the Agent Name and Instructions that we can set. 

You can change these settings to see how the differences in the agent's response. 

In [11]:
# Create the agent
agent = ChatCompletionAgent(
    service=chat_completion_service,
    name="TravelAgent",
    instructions="Answer questions about flight prices and seat availability for given destinations.",
    plugins=[FlightSimulation()],
)


## Running the Agent 

Now we will run the AI Agent.
In this snippet, we add two messages to user_inputs to demonstrate how the agent responds to follow-up questions.

The agent should call the correct function to get the flight prices for a given route and confirm the seat availability for that same route.

You can change the user_inputs to test different origins, destinations, and scenarios. 

In [12]:
user_inputs = [
    "What are the flight prices from New York to Tokyo?",
    "Is there seat availability from New York to Tokyo?",
    "Any flights available from Los Angeles to Paris?",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()
